# Document Analysis Assignment 1: Information Retrieval

In this assignment, your task is to index a new document collection into *Elasticsearch* and measure search performance based on predefined queries. 
A set of new documents collection containing more than 10000 goverment sites description and a set of predefined queries will be provided for this assignment.
Throughout this assginment, 
1. you will get better understanding of indexer including tokeniser, parser, and normalisers to improve the search performance given a predefined evaluation metric, 
2. you will get better understanding of search algorithm to obtain better search results, and 
3. you will find the best way to combine indexer and search algorithm to maximise the performance.

Below, you will solve five programming assignments (Q1-Q5), and three written assignments after that. We will check the correctness of your code, but the score of the programming assignments will be graded based on your performance on Kaggle competition.
- Write your code after `### Your code here`, and remove `raise NotImplementedError` after implementation.
- Written assignments should be written in the given notebook cells. Please write them direcly in to the designated cells, and upload the notebook file to Wattle page.
- Write answers in this notebook file, and upload the file to Wattle submission site. **Please rename and submit jupyter notebook file (`Assignment1.ipynb`) to `your_uid.ipynb` (e.g. `u6000001.ipynb`) with your written answers therein**. Do not upload any other files to Wattle except this notebook file.

For the Kaggle competition:
- Note that you are only allowed to upload **3 copies** of your results to Kaggle per day. Make every upload count, and don't waste your opportunities!

Score distribution (total 10 points)
- Kaggle competition (Q1-Q5): 4 points
- Written assignment 1: 2 points
- Written assignment 2: 2 points
- Written assignment 3: 2 points


## Coding assignment (Q1 - Q5), 4 points

## Q1. Index Gov dataset

Download a collection of government documents `gov.zip` from Wattle, and unzip the file. The unzipped file contains two sub-folders; `documents`, and `topics`. `documents` folder consists of sub-folders, each of which contains multiple documents. Your first job is to index the documents as we have done in the lab exercises.

- Note that depending on your machine, indexing may take several minutes to a few hours. You may implement multi-threaded version of indexing to mitigate the problem.

Below we provide the basic configuration for indexing.

In [1]:
# basic configuration for indexing
basic_settings = {
  "mappings": {
    "doc": {
      "properties": {
        "filename": {
          "type": "keyword",
          "index": False,
        },
        "path": {
          "type": "keyword",
          "index": False,
        },
        "text": {
          "type": "text",
          "similarity": "boolean",
          "analyzer": "my_analyzer",
          "search_analyzer": "my_analyzer"
        }
      }
    }
  },
  "settings": {
    "analysis": {
      "analyzer": {
        "my_analyzer": {
          "filter": [
            "stop"
          ],
          "char_filter": [
            "html_strip"
          ],
          "type": "custom",
          "tokenizer": "whitespace"
        }
      }
    }
  }   
}

You need to implement function `build_gov_index` below. Don't forget to remove `raise NotImplementedError` after implementation.

In [2]:
import os, io
from collections import namedtuple
from elasticsearch import Elasticsearch

ES_HOSTS = ['http://localhost:9200']
DOCS_PATH = 'gov/documents'
INDEX_NAME = 'gov'
DOC_TYPE = 'doc'

def build_gov_index(es_conn, index_name, doc_path, settings):
    # TODO implement function that 
    # 1. create index with `index_name`, if `index_name` already exists, remove the index first.
    # 2. index the documents under doc_path including subfolders into elasticsearch (hint: read demo carefully)
    # Note that this function will be used throughout this assignment    
    # YOUR CODE HERE
    Doc = namedtuple('Doc', 'filename path text')
    dataset=[]
    for root, dirs, files in os.walk(DOCS_PATH):
        for doc_path in files:
            
            filename = (root+"/"+doc_path).split('/')[-1]
            fp = io.open(root+"/"+doc_path, 'r', encoding = "ISO-8859-1")
            text = fp.read().strip()
            fp.close()
            dataset.append(Doc(filename = filename, text = text, path = doc_path))

            
            
    if es_conn.indices.exists(index_name):
        es_conn.indices.delete(index = index_name)
        print('index `{}` deleted'.format(index_name))
    es_conn.indices.create(index = index_name, ignore = 400, body = settings)
    print('index `{}` created'.format(index_name))            
    counter_read, counter_idx_failed = 0, 0 # counters

    # retrive & index documents
    for doc in dataset:
        res = es_conn.index(
            index = index_name,
            id = doc.filename,
            doc_type = DOC_TYPE,
            body = doc._asdict())
        counter_read += 1

        if res['result'] != 'created':
            conter_idx_failed += 1
        else:
            print('indexed {} documents'.format(counter_read))

    print('indexed {} docs to index `{}`, failed to index {} docs'.format(
        counter_read,
        index_name,
        counter_idx_failed
    ))
    
    # refresh after indexing
    es_conn.indices.refresh(index=index_name)  



es_conn = Elasticsearch(ES_HOSTS)
build_gov_index(es_conn, INDEX_NAME, DOCS_PATH, basic_settings)

## Q2. Search and measure performance

For the second task, you first need to read `topics/gov.topics` file. As we have done in lab demo session, each file of this file is formatted as

`query_id query_terms`

`query_id` is a numerical number, and `query_terms` consists of multiple keywords as search terms. Your job is to read the query file and search using the provided `search` function. You need to write the outputs of search results to `output.csv` file. The first line of `output.csv` file should start with the following header:

`QueryId,DocumentNumber,Similarity,Iteration,RunId,Rank`, 

but you only need to fill QueryId and DocumentNumber for this assignment. Specifically, for each query, you will

1. rank the output of search results based on their scores,
2. write top-10 documents to `output.csv` based on the highest score. Except `QueryID` and `DocumentNumber` the other fields should be filled with 0s. For example, each line of the output file will be formatted as:
    
    `1,G00-27-1804490,0,0,0,0`
    
    where `1` is `query_id`, and `G00-27-1804490` is the file name of retrived document (=`DocumentNumber`).
    
    White space is now allowe between fields, i.e. `1, G00-27-1804490, 0, 0, 0, 0` will not be evaluated properly. 
    - **Note that you are only allowed to write 10-documents at most for each query**. If your output file contains more than 10 documents per query, **you will get 0 score for the programming assignment**.
    
3. upload `output.csv` file to [kaggle competition site](#Upload-output-file-to-Kaggle-competition-site) (see the details below). Check the performance of the basic search algorithm in terms of Precision@10.

In [10]:
def search(query_string, es_conn, index_name):
    '''
        searches for query_string with default search algorithm
        input:
            - query_string: a query
            - es_conn: elasticsearch connection
            - index_name: name of index
        output:
            - a generator of tuple (filename, score)

    '''
    res = es_conn.search(index = index_name,
        body = {
            "_source": [ "filename"],
            "query": {
                "query_string": {
                    "query": query_string,
                }
            }
        }
    )
    return res['hits']['hits']

# TODO: read query file from `query_path`, search using `search_fn`,
#       and write top 10 outputs per query to `output_file`
#       Note that the function takes a search function as an argument, you can directly call the search function
#          as `result = search_fn(query_string, es_conn, index_name)` within the function.
#       This function will be used throughout this assignment
def read_search_write_output(search_fn, query_path, output_file):
    with open(output_file, 'w') as output:
        output.write('QueryId,DocumentNumber,Similarity,Iteration,RunId,Rank\n')  #for your convenience

        # YOUR CODE HERE
        
        f=io.open(query_path,'r',encoding = "ISO-8859-1")
        text=f.read().strip().split("\n")
        i=1
        for query in text:
            result = search_fn(query[2:], es_conn, INDEX_NAME)
            
            count=0
            for r in result:
                if count<10:
                    output.write(query[:2]+','+r['_source']['filename']+',0,0,0,0\n')
                    count=count+1
            i=i+1

                                
                        
                    
                    
        

query_path = 'gov/topics/gov.topics'
output_file = 'output.csv'
read_search_write_output(search, query_path, output_file)

## Q3. Improve indexer

You will be asked to change the configuration of indexer (`basic_settings`) to improve the search performance.

Please look at the elastic search official document [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis.html) for better understanding of configuration and other options.

Note that you can check how your tokeniser tokenises your input string via `analyze_query` function provided in the demo code.

In [4]:
# TODO: configure settings to define your own analyzer for indexing
q3_settings = {
  "mappings": {
    "doc": {
      "properties": {
        "filename": {
          "type": "keyword",
          "index": False,
        },
        "path": {
          "type": "keyword",
          "index": False,
        },
        "text": {
          "type": "text",
          "similarity": "boolean",
          "analyzer": "my_analyzer",
          "search_analyzer": "my_analyzer"
        }
      }
    }
  },
  "settings": {
    "analysis": {
      "analyzer": {
          "my_analyzer": {
            "filter": [
            "stop","lowercase","porter_stem"
          ],
          "char_filter": [
            "html_strip"
          ],
          "type": "custom",
          "tokenizer": "standard"
        }
      }
    }
  }
}

In [1]:
# TODO: run this block to generate an output based on q3_settings defined above.
build_gov_index(es_conn, INDEX_NAME, DOCS_PATH, q3_settings)
read_search_write_output(search, query_path, output_file)

NameError: name 'build_gov_index' is not defined

Upload the final output to Kaggle to check the difference in Precision@10.

## Q4. Improve search algorithm

*Elasticsearch* also provides multiple configurable scoring algorithms. For this task, you will be asked to find a better similarity module to improve the search performance. Please refer [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/similarity.html) for better understanding of configurable elasticsearch similarity modules.

You can also change the `search` function to improve performance. Please refer [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl.html) for better understanding of Query DSL used in *elasticsearch*.

In [12]:
# TODO: define your own analyzer for indexing and searching
q4_settings = {
  "mappings": {
    "doc": {
      "properties": {
        "filename": {
          "type": "keyword",
          "index": False,
        },
        "path": {
          "type": "keyword",
          "index": False,
        },
        "text": {
            # YOUR CODE HERE
            "type": "text",
            "similarity": "BM25",
            "analyzer": "my_analyzer",
            "search_analyzer": "my_analyzer"
        }
      }
    }
  },
  "settings": {
    "analysis": {
      "analyzer": {
            # YOUR CODE HERE
            "my_analyzer": {
            "filter": [
            "stop","lowercase","porter_stem"
          ],
          "char_filter": [
            "html_strip"
          ],
          "type": "custom",
          "tokenizer": "standard"
        }
      }
    }
  }
}


# TODO: change search algorithm to improve the search results, the return type should be the same as that of `search` function
def my_search(query_string, es_conn, index_name):
    # YOUR CODE HERE
    res = es_conn.search(index = index_name,
        body = {
            "_source": [ "filename"],
            "query": {
                "query_string": {
                    "query": query_string,
                }
            }
        }
    )
    return res['hits']['hits']



In [5]:
# TODO: run this block to generate an output based on q4_settings and my_search defined above.
build_gov_index(es_conn, INDEX_NAME, DOCS_PATH, q4_settings)
read_search_write_output(my_search, query_path, output_file)

index `gov` deleted
index `gov` created
indexed 1 documents
indexed 2 documents
indexed 3 documents
indexed 4 documents
indexed 5 documents
indexed 6 documents
indexed 7 documents
indexed 8 documents
indexed 9 documents
indexed 10 documents
indexed 11 documents
indexed 12 documents
indexed 13 documents
indexed 14 documents
indexed 15 documents
indexed 16 documents
indexed 17 documents
indexed 18 documents
indexed 19 documents
indexed 20 documents
indexed 21 documents
indexed 22 documents
indexed 23 documents
indexed 24 documents
indexed 25 documents
indexed 26 documents
indexed 27 documents
indexed 28 documents
indexed 29 documents
indexed 30 documents
indexed 31 documents
indexed 32 documents
indexed 33 documents
indexed 34 documents
indexed 35 documents
indexed 36 documents
indexed 37 documents
indexed 38 documents
indexed 39 documents
indexed 40 documents
indexed 41 documents
indexed 42 documents
indexed 43 documents
indexed 44 documents
indexed 45 documents
indexed 46 documents
ind

indexed 380 documents
indexed 381 documents
indexed 382 documents
indexed 383 documents
indexed 384 documents
indexed 385 documents
indexed 386 documents
indexed 387 documents
indexed 388 documents
indexed 389 documents
indexed 390 documents
indexed 391 documents
indexed 392 documents
indexed 393 documents
indexed 394 documents
indexed 395 documents
indexed 396 documents
indexed 397 documents
indexed 398 documents
indexed 399 documents
indexed 400 documents
indexed 401 documents
indexed 402 documents
indexed 403 documents
indexed 404 documents
indexed 405 documents
indexed 406 documents
indexed 407 documents
indexed 408 documents
indexed 409 documents
indexed 410 documents
indexed 411 documents
indexed 412 documents
indexed 413 documents
indexed 414 documents
indexed 415 documents
indexed 416 documents
indexed 417 documents
indexed 418 documents
indexed 419 documents
indexed 420 documents
indexed 421 documents
indexed 422 documents
indexed 423 documents
indexed 424 documents
indexed 42

indexed 754 documents
indexed 755 documents
indexed 756 documents
indexed 757 documents
indexed 758 documents
indexed 759 documents
indexed 760 documents
indexed 761 documents
indexed 762 documents
indexed 763 documents
indexed 764 documents
indexed 765 documents
indexed 766 documents
indexed 767 documents
indexed 768 documents
indexed 769 documents
indexed 770 documents
indexed 771 documents
indexed 772 documents
indexed 773 documents
indexed 774 documents
indexed 775 documents
indexed 776 documents
indexed 777 documents
indexed 778 documents
indexed 779 documents
indexed 780 documents
indexed 781 documents
indexed 782 documents
indexed 783 documents
indexed 784 documents
indexed 785 documents
indexed 786 documents
indexed 787 documents
indexed 788 documents
indexed 789 documents
indexed 790 documents
indexed 791 documents
indexed 792 documents
indexed 793 documents
indexed 794 documents
indexed 795 documents
indexed 796 documents
indexed 797 documents
indexed 798 documents
indexed 79

indexed 1124 documents
indexed 1125 documents
indexed 1126 documents
indexed 1127 documents
indexed 1128 documents
indexed 1129 documents
indexed 1130 documents
indexed 1131 documents
indexed 1132 documents
indexed 1133 documents
indexed 1134 documents
indexed 1135 documents
indexed 1136 documents
indexed 1137 documents
indexed 1138 documents
indexed 1139 documents
indexed 1140 documents
indexed 1141 documents
indexed 1142 documents
indexed 1143 documents
indexed 1144 documents
indexed 1145 documents
indexed 1146 documents
indexed 1147 documents
indexed 1148 documents
indexed 1149 documents
indexed 1150 documents
indexed 1151 documents
indexed 1152 documents
indexed 1153 documents
indexed 1154 documents
indexed 1155 documents
indexed 1156 documents
indexed 1157 documents
indexed 1158 documents
indexed 1159 documents
indexed 1160 documents
indexed 1161 documents
indexed 1162 documents
indexed 1163 documents
indexed 1164 documents
indexed 1165 documents
indexed 1166 documents
indexed 116

indexed 1484 documents
indexed 1485 documents
indexed 1486 documents
indexed 1487 documents
indexed 1488 documents
indexed 1489 documents
indexed 1490 documents
indexed 1491 documents
indexed 1492 documents
indexed 1493 documents
indexed 1494 documents
indexed 1495 documents
indexed 1496 documents
indexed 1497 documents
indexed 1498 documents
indexed 1499 documents
indexed 1500 documents
indexed 1501 documents
indexed 1502 documents
indexed 1503 documents
indexed 1504 documents
indexed 1505 documents
indexed 1506 documents
indexed 1507 documents
indexed 1508 documents
indexed 1509 documents
indexed 1510 documents
indexed 1511 documents
indexed 1512 documents
indexed 1513 documents
indexed 1514 documents
indexed 1515 documents
indexed 1516 documents
indexed 1517 documents
indexed 1518 documents
indexed 1519 documents
indexed 1520 documents
indexed 1521 documents
indexed 1522 documents
indexed 1523 documents
indexed 1524 documents
indexed 1525 documents
indexed 1526 documents
indexed 152

...

{u'_score': 14.816673, u'_type': u'doc', u'_id': u'G00-59-2927976', u'_source': {u'filename': u'G00-59-2927976'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 14.500883, u'_type': u'doc', u'_id': u'G00-97-3182735', u'_source': {u'filename': u'G00-97-3182735'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 13.868107, u'_type': u'doc', u'_id': u'G00-70-0954490', u'_source': {u'filename': u'G00-70-0954490'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 13.37429, u'_type': u'doc', u'_id': u'G00-65-4152139', u'_source': {u'filename': u'G00-65-4152139'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 12.382233, u'_type': u'doc', u'_id': u'G00-42-1312823', u'_source': {u'filename': u'G00-42-1312823'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 12.299551, u'_type': u'doc', u'_id': u'

{u'_score': 39.278145, u'_type': u'doc', u'_id': u'G00-27-1492903', u'_source': {u'filename': u'G00-27-1492903'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 36.785217, u'_type': u'doc', u'_id': u'G00-02-3479535', u'_source': {u'filename': u'G00-02-3479535'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 36.409737, u'_type': u'doc', u'_id': u'G00-28-2516057', u'_source': {u'filename': u'G00-28-2516057'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 34.656124, u'_type': u'doc', u'_id': u'G00-15-1460278', u'_source': {u'filename': u'G00-15-1460278'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 33.144577, u'_type': u'doc', u'_id': u'G00-94-4070183', u'_source': {u'filename': u'G00-94-4070183'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 32.763412, u'_type': u'doc', u'_id': u

{u'_score': 14.918914, u'_type': u'doc', u'_id': u'G00-71-2945298', u'_source': {u'filename': u'G00-71-2945298'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 14.780872, u'_type': u'doc', u'_id': u'G00-74-0010893', u'_source': {u'filename': u'G00-74-0010893'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 14.648048, u'_type': u'doc', u'_id': u'G00-93-0926003', u'_source': {u'filename': u'G00-93-0926003'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 14.195723, u'_type': u'doc', u'_id': u'G00-44-2915286', u'_source': {u'filename': u'G00-44-2915286'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 14.1798, u'_type': u'doc', u'_id': u'G00-34-2274617', u'_source': {u'filename': u'G00-34-2274617'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 14.133022, u'_type': u'doc', u'_id': u'G

--------------------------------------------------------
{u'_score': 17.936409, u'_type': u'doc', u'_id': u'G00-49-0000000', u'_source': {u'filename': u'G00-49-0000000'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 17.917503, u'_type': u'doc', u'_id': u'G00-85-3015991', u'_source': {u'filename': u'G00-85-3015991'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 17.500185, u'_type': u'doc', u'_id': u'G00-18-2342428', u'_source': {u'filename': u'G00-18-2342428'}, u'_index': u'gov'}
--------------------------------------------------------
National
{u'_score': 18.197721, u'_type': u'doc', u'_id': u'G00-67-0152545', u'_source': {u'filename': u'G00-67-0152545'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 16.842161, u'_type': u'doc', u'_id': u'G00-83-1887140', u'_source': {u'filename': u'G00-83-1887140'}, u'_index': u'gov'}
-----------------------------------------

{u'_score': 17.931456, u'_type': u'doc', u'_id': u'G00-17-2226096', u'_source': {u'filename': u'G00-17-2226096'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 17.832096, u'_type': u'doc', u'_id': u'G00-05-0295089', u'_source': {u'filename': u'G00-05-0295089'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 17.405172, u'_type': u'doc', u'_id': u'G00-77-0056278', u'_source': {u'filename': u'G00-77-0056278'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 17.066643, u'_type': u'doc', u'_id': u'G00-94-0708359', u'_source': {u'filename': u'G00-94-0708359'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 16.891617, u'_type': u'doc', u'_id': u'G00-09-0131668', u'_source': {u'filename': u'G00-09-0131668'}, u'_index': u'gov'}
--------------------------------------------------------
{u'_score': 16.703802, u'_type': u'doc', u'_id': u

## Q5. Find the best combination

Now it's time to explorer the best configuration of indexer and search algorithms. Each combination will yield a different search outcome. Try different combinations and report best results below.

In [ ]:
# TODO: find the best combination of indexer configuration and search algorithm to maximise the performance of search result.
best_settings = {
    # YOUR CODE HERE
    "mappings": {
    "doc": {
      "properties": {
        "filename": {
          "type": "keyword",
          "index": False,
        },
        "path": {
          "type": "keyword",
          "index": False,
        },
        "text": {
            # YOUR CODE HERE
            "type": "text",
            "similarity": "BM25",
            "analyzer": "my_analyzer",
            "search_analyzer": "my_analyzer"
        }
      }
    }
  },
  "settings": {
    "analysis": {
      "analyzer": {
            # YOUR CODE HERE
            "my_analyzer": {
            "filter": [
            "stop","lowercase","porter_stem","shingle"
          ],
          "char_filter": [
            "html_strip"
          ],
          "type": "custom",
          "tokenizer": "standard"
        }
      }
    }
  }
}



def best_search(query_string, es_conn, index_name):
    # YOUR CODE HERE
    res = es_conn.search(index = index_name,
        body = {
            "_source": [ "filename"],
            "query": {
                "query_string": {
                    "query": query_string,
                }
            }
        }
    )
    return res['hits']['hits']


In [ ]:
# TODO: run this block to generate the output
build_gov_index(es_conn, INDEX_NAME, DOCS_PATH, best_settings)
read_search_write_output(my_search, query_path, output_file)

Answer following questions based on your implementation from Q1 to Q5.

# Written Assignment 1 (2pt)

What changes did you make for the **indexer** to improve the performance of the system? Why do you think it improves the performance?

(provide answer using bullet list with 2~3 items (Check [this](https://sourceforge.net/p/jupiter/wiki/markdown_syntax/#md_ex_lists) if you are not familiar with markdown syntax))

YOUR ANSWER HERE
1. Tokenizer was changed to 'standard' 
2. Token filters were added like 'lowercase','porter_stem','shingle'
3. Changing the tokenizer to standard helped in removing unnecessary punctuations and making the tokens more general. Adding token filters normalized the tokens. Shingle filter helped in matching phrases and helping in increasing the efficiency of the returned document set

# Written Assignment 2  (2pt)

What changes did you make for the **search algorithm** to improve the performance of the system? Why do you think it improves the performance?

(provide answer using bullet list with 2~3 items)

YOUR ANSWER HERE
1. The similarity was changed to BM25 from boolean.
2. BM25 uses a version of TF/IDF for scoring the returned documents which is much better than the boolean similarity. Boolean similarity would just assign a constant score to all the matched documents.

# Written Assignment 3 (2pt)

In the Kaggle competition, we only use Precision@10 as an evaluation metric. What other metrics can be used to measure the performance of IR system for the government document collection? Provide two metrics and explain why.

(provide answer using bullet list with 2~3 items)

YOUR ANSWER HERE
1. Recall : because it tells us the fraction of relevant documents that were retrieved. Hence, we can improve the recall factor to make our system more efficient.
2. F-Score : A good IR system would have a balance between precision and recall. F-Score gives us the harmonic mean of precision and recall. Hence, it would be most suited to help us design a good IR system.

# Upload output file to Kaggle competition site

Once you generate `output.csv` file, you can upload your result on Kaggle competition site. To upload and evaluate your result

1. Go to Kaggle competition site: [Click here](https://www.kaggle.com/t/26d21af1d3ea4447b86737fe889160ff).
1. Sign up for Kaggle if you do not have an account. Go back to the [original kaggle page](https://www.kaggle.com/t/26d21af1d3ea4447b86737fe889160ff).
1. Before submitting the result, first go to `team` manu and change **your team name as your university id**.
![ChangeUID](images/changeuid.png)
1. Time to submit your own result. Click `submit predictions` in the manu, you may need to agree the competetion rules before submitting your result.
1. Upload your output csv file, you can write additional description of your submission in the description box.
    Note that you are only allowed to submit **3 results per day**. Do not upload an arbitrary result and think which algorithm or parser will perform the best.
1. If your output format is correct, the system will generate your score automatically.
1. Go to `Leaderboard` menu. The leaderboard will show the current score of the other students.
![Leaderboard](images/leaderboard.png)


Note that you can check all of your submission from `my submission` menu. Please select one best performing submission before the assignment due. The selected submission will be used to measure the performance of *hidden* test case (see below for details).
![Check](images/check.png)

### Evaluation

The system uses *precision@10* to measure the performance of each submission, which basically measure the accuracy of your top 10 returned documents.

It is important to understand that the leaderboard score will be only computed based on the half of the test cases, and the remaining half will be computed after the deadline based on your selected submission. This process will ensure that your performance is not only applicable for the known test cases, but also generalised to the unknown test cases. We will combine these two performances to score the first assignment.